<a href="https://colab.research.google.com/github/pinedance/gym-AI-NLP/blob/main/gym_word_embeddings_with_BERT_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings with BERT

Chris McCormick이 만든 [이 코드](https://github.com/pinedance/gym-AI-NLP/blob/main/gym_word_embeddings_with_BERT.ipynb)를 Class로 만들어 개량하였다.

REF
* Chris McCormick의 Tutorial: [BERT Word Embeddings Tutorial](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/)
* [Getting Contextualized Word Embeddings with BERT](https://medium.com/@r3d_robot/getting-contextualized-word-embeddings-with-bert-20798d8b43a4)
* [3 Types of Contextualized Word Embeddings From BERT Using Transfer Learning](https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d)



In [1]:
import torch
from transformers import BertTokenizer, BertModel

import pandas as pd
import numpy as np
import nltk

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Loading the pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True, )
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [4]:
# Setting up the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
class qModel():

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def prepare_text( self, text ):
        """
        Preprocesses text input in a way that BERT can interpret.
        """
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids( tokenized_text )
        segments_ids = [1] * len( indexed_tokens )

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor( [ indexed_tokens ] )
        segments_tensors = torch.tensor( [ segments_ids ] )

        self.text = text
        self.marked_text = marked_text
        self.tokenized_text = tokenized_text
        self.indexed_tokens = indexed_tokens
        self.segments_ids = segments_ids
        self.tokens_tensor = tokens_tensor
        self.segments_tensors = segments_tensors
        if hasattr(self, 'outputs'):
            del self.outputs

        return self

    def inference( self, text=None ):
        """
        Get embeddings from an embedding model
        """
        # Gradient calculation id disabled
        # Model is in inference mode

        if text is not None:
            self.prepare_text( text )

        if not hasattr(self, 'text'):
            raise RuntimeError('text가 준비되지 않았습니다. text 값을 입력해 주세요.')

        with torch.no_grad():
            self.outputs = self.model( self.tokens_tensor, self.segments_tensors )

        return self

    def calc_token_embeddings(self, text=None ):
        """
        Preprocesses text input in a way that BERT can interpret.
        """

        if text is not None:
            self.inference( text )

        if not hasattr(self, 'outputs'):
            raise RuntimeError('outputs이 준비되지 않았습니다. text 값을 입력해 주세요.')

        hidden_states = self.outputs.hidden_states
        # Concatenate the tensors for all layers. We use `stack` here to
        # create a new dimension in the tensor.
        token_embeddings = torch.stack(hidden_states, dim=0)
        # Remove dimension 1, the "batches".
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        # Swap dimensions 0 and 1.
        token_embeddings = token_embeddings.permute(1,0,2)
        self.token_embeddings = token_embeddings
        return self

    def calc_token_vectors(self, text=None):
        """
        Preprocesses text input in a way that BERT can interpret.
        """

        if text is not None:
            self.calc_token_embeddings( text )

        if not hasattr(self, 'token_embeddings'):
            self.calc_token_embeddings()

        # intialized list to store embeddings
        token_vecs_cat = []
        token_vecs_sum = []

        # "token_embeddings" is a [Y x 12 x 768] tensor
        # where Y is the number of tokens in the sentence
        # loop over tokens in sentence
        for token in self.token_embeddings:
            # "token" is a [12 x 768] tensor

            # Method1: concat last 4 layers
            cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
            token_vecs_cat.append(cat_vec)

            # Method2: sum the vectors from the last four layers
            sum_vec = torch.sum(token[-4:], dim=0)
            token_vecs_sum.append(sum_vec)

        self.token_vectors = token_vecs_cat
        self.token_vectors_sum = token_vecs_sum

        return self

    def calc_sentence_vectors(self, text=None):
        """
        Preprocesses text input in a way that BERT can interpret.
        """
        if text is not None:
            self.calc_token_vectors( text )

        if not hasattr(self, 'token_vectors'):
            raise RuntimeError('token_vectors가 준비되지 않았습니다. "calc_token_vectors" 명령을 먼저 수행하세요.')

        token_vecs = self.outputs.hidden_states[-2][0]
        # Calculate the average of all 22 token vectors.
        sentence_embedding = torch.mean(token_vecs, dim=0)
        self.sentence_vectors = sentence_embedding
        return self


    """
    Preprocesses text input in a way that BERT can interpret.
    """

    def get_outputs(self):
        """
        Preprocesses text input in a way that BERT can interpret.
        """
        if not hasattr(self, 'outputs'):
            raise RuntimeError('outputs이 준비되지 않았습니다. "inference" 명령을 먼저 수행하세요.')

        return self.outputs

    def get_hidden_states(self):
        """
        Preprocesses text input in a way that BERT can interpret.
        """
        if not hasattr(self, 'outputs'):
            raise RuntimeError('outputs이 준비되지 않았습니다. "inference" 명령을 먼저 수행하세요.')

        # outputs[2] == outputs.hidden_states
        # outputs[2][-1] == outputs.last_hidden_states
        return self.outputs.hidden_states

    def get_token_vectors(self, text=None, method="cat"):
        """
        Preprocesses text input in a way that BERT can interpret.
        """
        if text is not None:
            self.calc_sentence_vectors( text )

        if not hasattr(self, 'token_vectors'):
            raise RuntimeError('token_vectors가 준비되지 않았습니다. "calc_token_vectors" 명령을 먼저 수행하세요.')

        if method == "sum":
            return self.token_vectors_sum
        else:
            return self.token_vectors

    def get_sentence_vectors(self, text=None):
        """
        Preprocesses text input in a way that BERT can interpret.
        """
        if text is not None:
            self.calc_sentence_vectors( text )

        if not hasattr(self, 'sentence_vectors'):
            raise RuntimeError('sentence_vectors가 준비되지 않았습니다. "calc_sentence_vectors" 명령을 먼저 수행하세요.')

        return self.sentence_vectors


    """
    Preprocesses text input in a way that BERT can interpret.
    """

    def fit( self, text ):
        self.inference( text )
        return self

    def transform( self ):
        self.calc_sentence_vectors()
        return self.sentence_vectors

    def fit_transform( self, text ):
        self.calc_sentence_vectors( text )
        return self.sentence_vectors

In [13]:
text = "After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."
mymodel = qModel( model, tokenizer )
sentence_vec = mymodel.fit_transform( text )

In [9]:
for i, token_str in enumerate( mymodel.tokenized_text ) :
    print (i, token_str)

0 [CLS]
1 after
2 stealing
3 money
4 from
5 the
6 bank
7 vault
8 ,
9 the
10 bank
11 robber
12 was
13 seen
14 fishing
15 on
16 the
17 mississippi
18 river
19 bank
20 .
21 [SEP]


In [10]:
token_vector = mymodel.token_vectors
print('First 5 vector values for each instance of "bank".')
print('')
print("bank vault   ", str(token_vector[6][:5]))
print("bank robber  ", str(token_vector[10][:5]))
print("river bank   ", str(token_vector[19][:5]))

First 5 vector values for each instance of "bank".

bank vault    tensor([ 0.9001, -0.5380, -0.1669,  0.2242,  0.6897])
bank robber   tensor([ 0.7977, -0.5217, -0.1984,  0.1890,  0.5941])
river bank    tensor([ 0.2961, -0.2856, -0.0382,  0.1674,  0.7713])


In [11]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vector[10], token_vector[19])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vector[10], token_vector[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.94
Vector similarity for *different* meanings:  0.69


In [12]:
sentences = ["bank",
	"he eventually sold the shares back to the bank at a premium.",
	"the bank strongly resisted cutting interest rates.",
	"the bank will supply and buy back foreign currency.",
	"the bank is pressing us for repayment of the loan.",
	"the bank left its lending rates unchanged.",
	"the river flowed over the bank.",
	"tall, luxuriant plants grew along the river bank.",
	"his soldiers were arrayed along the river bank.",
	"wild flowers adorned the river bank.",
	"two fox cubs romped playfully on the river bank.",
	"the jewels were kept in a bank vault.",
	"you can stow your jewellery away in the bank.",
	"most of the money was in storage in bank vaults.",
	"the diamonds are shut away in a bank vault somewhere.",
	"thieves broke into the bank vault.",
	"can I bank on your support?",
	"you can bank on him to hand you a reasonable bill for your services.",
	"don't bank on your friends to help you out of trouble.",
	"you can bank on me when you need money.",
	"i bank on your help."
]

In [14]:
from collections import OrderedDict

context_embeddings = []
context_tokens = []

emodel = qModel( model, tokenizer )

for sentence in sentences:

    sentence_vec = emodel.fit_transform( text )

    # make ordered dictionary to keep track of the position of each   word
    tokens = OrderedDict()
    tokenized_text = emodel.tokenized_text
    list_token_embeddings = emodel.token_embeddings

    # loop over tokens in sensitive sentence
    for token in tokenized_text[1:-1]:
        # keep track of position of word and whether it occurs multiple times
        if token in tokens:
            tokens[token] += 1
        else:
            tokens[token] = 1

        # compute the position of the current token
        token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
        current_index = token_indices[tokens[token]-1]
        # get the corresponding embedding
        token_vec = list_token_embeddings[current_index]

        # save values
        context_tokens.append(token)
        context_embeddings.append(token_vec)

In [15]:
context_embeddings

[tensor([[ 0.2329,  0.1390,  0.2979,  ..., -0.0655,  0.8885,  0.5109],
         [ 0.3819,  0.1475,  0.2414,  ...,  0.3397,  0.7607,  0.4999],
         [ 0.1430,  0.0747,  0.0595,  ..., -0.2914,  0.1733,  0.4265],
         ...,
         [-0.3215, -1.1213, -0.2349,  ..., -0.5940,  0.1851, -0.5076],
         [-0.2817, -0.6142, -0.4499,  ..., -0.7273,  0.0304, -0.5106],
         [-0.1323, -0.2762, -0.3495,  ..., -0.4567,  0.3786, -0.1096]]),
 tensor([[ 0.2257, -0.7165, -0.7255,  ...,  0.4844,  0.6030, -0.0957],
         [ 0.1705, -0.6168, -0.7296,  ...,  0.8631,  0.6274, -0.3727],
         [ 0.5752, -1.2385, -0.5650,  ...,  0.8995,  0.4652, -0.8080],
         ...,
         [-0.4795, -1.0745, -0.1744,  ..., -0.5026,  0.1293, -0.1963],
         [-0.4519, -0.2628, -0.1917,  ..., -0.5405, -0.2146, -0.2140],
         [-0.3626, -0.4002,  0.0676,  ..., -0.3207, -0.2709, -0.3004]]),
 tensor([[-0.6828, -0.6997, -0.3595,  ...,  0.7534,  0.8656, -0.0328],
         [-0.1935, -0.8975, -0.5827,  ...,  0